# Data enrichment

In [40]:
import pandas as pd
import numpy as np
import re
import string

In [41]:
info_saber    = pd.read_csv("/home/laura/Documents/DS4A/Project/src/data/Coleg_Saber.csv", engine='python', sep = ',')
norm_fields = [ field for field in info_saber.columns if re.search('Norm',field)]
rank_fields = [('PUNT_SOCIALES_CIUDADANAS_Norm', 'sociales_ciudanas_rank'),
               ('PUNT_C_NATURALES_Norm','c_naturales_rank'),
               ('PUNT_MATEMATICAS_Norm','matematicas_rank'),
               ('PUNT_LECTURA_CRITICA_Norm','lectura_critica_rank'),
               ('PUNT_INGLES_Norm',  'ingles_rank'),
                ]
for norm, rank in rank_fields: 
    info_saber[rank] = info_saber[norm_fields].rank(axis=1, method='first', ascending=False)[norm]
print(info_saber.columns)
info_saber.drop(columns =['COLE_COD_MCPIO_UBICACION', 'COLE_CALENDARIO',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_MATEMATICAS',
       'PUNT_LECTURA_CRITICA', 'PUNT_INGLES', 'PUNT_SOCIALES_CIUDADANAS_Norm',
       'PUNT_C_NATURALES_Norm', 'PUNT_MATEMATICAS_Norm',
       'PUNT_LECTURA_CRITICA_Norm', 'PUNT_INGLES_Norm'], inplace=True )
info_saber.head(3)

Index(['COLE_COD_MCPIO_UBICACION', 'COLE_COD_DANE_ESTABLECIMIENTO',
       'COLE_CALENDARIO', 'COLE_JORNADA', 'PUNT_GLOBAL',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_MATEMATICAS',
       'PUNT_LECTURA_CRITICA', 'PUNT_INGLES', 'PUNT_SOCIALES_CIUDADANAS_Norm',
       'PUNT_C_NATURALES_Norm', 'PUNT_MATEMATICAS_Norm',
       'PUNT_LECTURA_CRITICA_Norm', 'PUNT_INGLES_Norm',
       'sociales_ciudanas_rank', 'c_naturales_rank', 'matematicas_rank',
       'lectura_critica_rank', 'ingles_rank'],
      dtype='object')


,COLE_COD_DANE_ESTABLECIMIENTO,COLE_CALENDARIO,PUNT_GLOBAL,sociales_ciudanas_rank,c_naturales_rank,matematicas_rank,lectura_critica_rank,ingles_rank
0,105001000167,A,238.0,5.0,3.0,2.0,1.0,4.0
1,105001007919,A,252.5,2.0,5.0,3.0,1.0,4.0
2,105001010928,A,282.0,3.0,4.0,5.0,1.0,2.0


In [42]:

# 'direccion', 'telefono','zona', 'niveles', 'caracter', 'especialidad',
# 'idiomas', 'correo electronico'

#  'modelos educativos' --> (jovenes extraedad y adultos, tradicional), 'capacidades excepcionales',
#  'discapacidades','matricula contratada','estrato socio-economico',

#        'nombre rector', 'tipo establecimiento', 'etnias', 'sector' --> (oficial, no oficial),'genero'
#        'licencia', 'grados', 'numero de sedes', 'estado','prestador de servicio', 'propiedad de la planta fisica', 'resguardo',
#         'internado',
#         

# , 'TOTAL_ESTU', 'COLE_NATURALEZA',
#    'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_NOMBRE_SEDE',
#    'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION',

In [43]:
def normalize_name(txt1):
    if(pd.isna(txt1)):
        return ""
    clean_name = txt1.lower()
    remove_accents = clean_name.maketrans("áéíóúüñ", "aeiouun")
    clean_name = clean_name.translate(remove_accents)
    clean_name = clean_name.strip()
    return clean_name

In [44]:
school_data = pd.read_csv(f'clean_schools/clean_schools_joined.csv').drop(columns=['Unnamed: 0'])
school_data = school_data[school_data['Punt_global_mean_2021'].notna()]
school_data = school_data[school_data['COLE_CALENDARIO'] != 'OTRO']
school_data.replace(to_replace=r'COMPLETA|UNICA', value='COMPLETA_UNICA', regex=True, inplace=True)

In [45]:
sineb_data = pd.read_csv(f'clean_schools/colegios_establecimientos.csv', sep = ";", encoding='latin')
sineb_data.rename(normalize_name, axis='columns',inplace=True)
sineb_data = sineb_data[['codigo','nombre','direccion', 'telefono','zona', 'niveles', 'caracter', 'especialidad',
'idiomas', 'correo electronico','modelos educativos', 'capacidades excepcionales',
'discapacidades','matricula contratada','estrato socio-economico']]

In [39]:
rich_data = school_data.merge(sineb_data, how= "inner", left_on='COLE_COD_DANE_SEDE', right_on='codigo').drop(columns=['codigo', 'COLE_NOMBRE_SEDE'])
rich_data.rename(columns={'nombre':'COLE_NOMBRE_SEDE'}, inplace=True)
del school_data, sineb_data
print(rich_data.columns)
print(info_saber.columns)
rich_data = info_saber.merge(rich_data, how= "inner", left_on=['COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_JORNADA'], right_on=['COLE_COD_DANE_SEDE', 'COLE_JORNADA'])
rich_data[['COLE_COD_DANE_SEDE', 'COLE_NOMBRE_SEDE',
        'COLE_JORNADA', 
        'COLE_MCPIO_UBICACION',
       'COLE_DEPTO_UBICACION', 'COLE_GENERO', 'COLE_CALENDARIO',
       'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL',
       'COLE_AREA_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_COD_DEPTO_UBICACION', 
       'direccion', 'telefono',
       'zona', 'niveles', 'caracter', 'especialidad', 'idiomas',
       'correo electronico', 'modelos educativos', 'capacidades excepcionales',
       'discapacidades', 'matricula contratada', 'estrato socio-economico',
       'Punt_lectura_critica_mean_2017',
       'Punt_matematicas_mean_2017', 'Punt_c_naturales_mean_2017',
       'Punt_sociales_ciudadanas_mean_2017', 'Punt_ingles_mean_2017',
       'Punt_global_mean_2017', 'Punt_lectura_critica_mean_2018',
       'Punt_matematicas_mean_2018', 'Punt_c_naturales_mean_2018',
       'Punt_sociales_ciudadanas_mean_2018', 'Punt_ingles_mean_2018',
       'Punt_global_mean_2018', 'Punt_lectura_critica_mean_2019',
       'Punt_matematicas_mean_2019', 'Punt_c_naturales_mean_2019',
       'Punt_sociales_ciudadanas_mean_2019', 'Punt_ingles_mean_2019',
       'Punt_global_mean_2019', 'Punt_lectura_critica_mean_2020',
       'Punt_matematicas_mean_2020', 'Punt_c_naturales_mean_2020',
       'Punt_sociales_ciudadanas_mean_2020', 'Punt_ingles_mean_2020',
       'Punt_global_mean_2020', 'Punt_lectura_critica_mean_2021',
       'Punt_matematicas_mean_2021', 'Punt_c_naturales_mean_2021',
       'Punt_sociales_ciudadanas_mean_2021', 'Punt_ingles_mean_2021',
       'Punt_global_mean_2021']]

Index(['COLE_COD_DANE_SEDE', 'COLE_JORNADA', 'COLE_MCPIO_UBICACION',
       'COLE_DEPTO_UBICACION', 'COLE_GENERO', 'COLE_CALENDARIO',
       'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL',
       'COLE_AREA_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_COD_DEPTO_UBICACION', 'Punt_lectura_critica_mean_2017',
       'Punt_matematicas_mean_2017', 'Punt_c_naturales_mean_2017',
       'Punt_sociales_ciudadanas_mean_2017', 'Punt_ingles_mean_2017',
       'Punt_global_mean_2017', 'Punt_lectura_critica_mean_2018',
       'Punt_matematicas_mean_2018', 'Punt_c_naturales_mean_2018',
       'Punt_sociales_ciudadanas_mean_2018', 'Punt_ingles_mean_2018',
       'Punt_global_mean_2018', 'Punt_lectura_critica_mean_2019',
       'Punt_matematicas_mean_2019', 'Punt_c_naturales_mean_2019',
       'Punt_sociales_ciudadanas_mean_2019', 'Punt_ingles_mean_2019',
       'Punt_global_mean_2019', 'Punt_lectura_critica_mean_2020',
       'Punt_matematicas_mean_2020', 'Punt_c_naturales_mean_2020',

KeyError: 'COLE_JORNADA'

In [ ]:
rich_data.head(3)

,COLE_COD_DANE_ESTABLECIMIENTO,PUNT_GLOBAL,sociales_ciudanas_rank,c_naturales_rank,matematicas_rank,lectura_critica_rank,ingles_rank,COLE_COD_DANE_SEDE,COLE_JORNADA,COLE_MCPIO_UBICACION,...,niveles,caracter,especialidad,idiomas,correo electronico,modelos educativos,capacidades excepcionales,discapacidades,matricula contratada,estrato socio-economico
0,105001007919,252.5,2.0,5.0,3.0,1.0,4.0,105001007919,MAÑANA,MEDELLÍN,...,"PREESCOLAR,MEDIA,BÁSICA SECUNDARIA,BÁSICA PRIM...",TÉCNICO/ACADEMICO,"OTRO,COMERCIAL,ACADÉMICA",NaN,sagradafamiliahpar@yahoo.es,EDUCACIÓN TRADICIONAL,NaN,NaN,SI,NaN
1,105001010928,282.0,3.0,4.0,5.0,1.0,2.0,105001010928,COMPLETA_UNICA,MEDELLÍN,...,"PREESCOLAR,MEDIA,BÁSICA SECUNDARIA,BÁSICA PRIM...",ACADÉMICO,ACADÉMICA,NaN,luferbet@yahoo.es lerincedbos240@hotmail.com,EDUCACIÓN TRADICIONAL,NaN,NaN,NO,NaN
2,305001000077,281.0,5.0,4.0,3.0,2.0,1.0,305001000077,COMPLETA_UNICA,MEDELLÍN,...,"PREESCOLAR,MEDIA,BÁSICA SECUNDARIA,BÁSICA PRIM...",ACADÉMICO,ACADÉMICA,NaN,correo@thenewschool.edu.co,EDUCACIÓN TRADICIONAL,NaN,NaN,NO,NaN


In [ ]:
info_saber.loc[96:100,[ field for field in info_saber.columns if re.search('Norm|rank',field)]]

,sociales_ciudanas_rank,c_naturales_rank,matematicas_rank,lectura_critica_rank,ingles_rank
96,2.0,3.0,4.0,1.0,5.0
97,2.0,4.0,3.0,1.0,5.0
98,4.0,5.0,2.0,3.0,1.0
99,3.0,4.0,2.0,5.0,1.0
100,2.0,5.0,3.0,1.0,4.0


In [ ]:
rich_data.to_csv('final_schools.csv')